<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/simulationTEG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Simulation of a TEG dehdyration process
In this notebook we will establish code for using NeqSim for simulation of a TEG dehdyration process.

In [2]:
# Setting up the Colab page to use neqsim
%%capture
!pip install neqsim
import neqsim
from neqsim.thermo.thermoTools import *

# Seting up the process in Python code
....

In [31]:
#Input parameters
inputData = {
  "feedGasFlowRate": (117000.0+90000.0)*24.0/1.0e6,
  "feedGasTemperature": 30.0,
  "feedGasPressure": 44.0,
  "absorberFeedGasTemperature": 30.7,
  "absorberFeedGasPressure": 44.0,
  "leanTEGFlowRate": 5932.0,
  "leanTEGTemperature": 36.2,
  "flashDrumPressure": 6.0,
  "reboilerPressure": 1.3,
  "condenserTemperature": 94.0,
  "condenserPressure": 1.2,
  "reboilerTemperature": 201.0,
  "strippingGasRate": 300.0,
  "strippingGasFeedTemperature": 200.0,
  "regenerationGasCoolerTemperature": 35.0,
  "TEGpumpIsentropicEfficiency": 0.75,
  'finefilterdeltaP': 0.01,
  'carbonfilterdeltaP': 0.01,
  "numberOfEquilibriumStagesTEGabsorber": 4,
  "stageEfficiencyTEGabsorber": 0.53,
  "stageEfficiencyStripper": 0.5,
  "numberOfEquilibriumStagesStripper": 7,
  "UAvalueLeanRichTEGHeatExchanger": 16000.0
}

In [40]:
from neqsim.thermo import fluid
from neqsim import methods
from neqsim.process import clearProcess, heater, stream, runProcess, stream, saturator, valve, filters, heatExchanger, simpleTEGAbsorber

# Start by creating a fluid in neqsim
feedGas = fluid("cpa")  # create a fluid using the SRK-Eo
feedGas.addComponent("nitrogen", 1.0)
feedGas.addComponent("CO2", 2.0)
feedGas.addComponent("methane", 85.0)
feedGas.addComponent("ethane", 5.0)
feedGas.addComponent("propane", 3.0)
feedGas.addComponent("i-butane", 2.0)
feedGas.addComponent("n-butane", 2.0)
feedGas.addComponent("water", 0.0)
feedGas.addComponent("TEG", 0.0)
feedGas.setMixingRule(10)

dryFeedGas = stream(feedGas)
dryFeedGas.setName('dry feed gas')
dryFeedGas.setFlowRate(inputData['feedGasFlowRate'], 'MSm3/day')
dryFeedGas.setTemperature(inputData['feedGasTemperature'], 'C')
dryFeedGas.setPressure(inputData['feedGasPressure'], 'bara')

saturatedFeedGas = saturator(dryFeedGas)
saturatedFeedGas.setName("water saturator")

waterSaturatedFeedGas = stream(saturatedFeedGas.getOutStream())
waterSaturatedFeedGas.setName("water saturated feed gas");

feedTEG = feedGas.clone();
feedTEG.setMolarComposition([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015, 0.985])

feedTPsetterToAbsorber = heater(waterSaturatedFeedGas)
feedTPsetterToAbsorber.setOutPressure(inputData['absorberFeedGasPressure'], "bara")
feedTPsetterToAbsorber.setOutTemperature(inputData['absorberFeedGasTemperature'], "C")

feedToAbsorber = stream(feedTPsetterToAbsorber.getOutStream())
feedToAbsorber.setName("feed to TEG absorber")

TEGFeed = stream(feedTEG)
TEGFeed.setName('lean TEG to absorber')
#TEGFeed.setFlowRate(inputData['feedGasFlowRate'], 'MSm3/day')
#TEGFeed.setFlowRate(inputData['leanTEGFlowRate'], 'kg/hr')
TEGFeed.setTemperature(inputData['leanTEGTemperature'], 'C')
TEGFeed.setPressure(inputData['absorberFeedGasPressure'], 'bara')

absorber = simpleTEGAbsorber()
absorber.setName("TEG absorber")
absorber.addGasInStream(feedToAbsorber)
absorber.addSolventInStream(TEGFeed)
absorber.setNumberOfStages(inputData['numberOfEquilibriumStagesTEGabsorber'])
absorber.setStageEfficiency(inputData['stageEfficiencyTEGabsorber'])
absorber.setInternalDiameter(1.7)

dehydratedGas = stream(absorber.getGasOutStream());
dehydratedGas.setName("dry gas from absorber")

richTEG = stream(absorber.getSolventOutStream())
richTEG.setName("rich TEG from absorber")

glycol_flash_valve = valve(richTEG)
glycol_flash_valve.setName("Rich TEG HP flash valve")
glycol_flash_valve.setOutletPressure(inputData['flashDrumPressure'])

richGLycolHeaterCondenser = heater(glycol_flash_valve.getOutStream())
richGLycolHeaterCondenser.setName("rich TEG preheater");

flashSep = separator(richGLycolHeaterCondenser.getOutStream())
flashSep.setName("degasing separator")

flashGas = stream(flashSep.getGasOutStream())
flashGas.setName("gas from degasing separator")

flashLiquid = stream(flashSep.getLiquidOutStream())
flashLiquid.setName("liquid from degasing separator")

fineFilter = filters(flashLiquid)
fineFilter.setName("TEG fine filter")
fineFilter.setDeltaP(inputData['finefilterdeltaP'], "bara")

heatEx = new heatExchanger(carbonFilter.getOutStream())
heatEx.setName("lean/rich TEG heat-exchanger")
heatEx.setGuessOutTemperature(273.15 + 130.0)
heatEx.setUAvalue(inputData['UAvalueLeanRichTEGHeatExchanger'])



java.lang.RuntimeException: ignored